In [15]:
import pandas as pd
import numpy as np

In [16]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [20]:
concrete_data.shape

(1030, 9)

In [21]:
# split the data into predectors and target
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [22]:
# normalize the data 
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [23]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [24]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(8,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
# split the data into test and training
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

In [27]:
# build the model
model = regression_model()

In [28]:
# fit the model
epochs = 100
model.fit(X_train, y_train, epochs=epochs, verbose=2)

Epoch 1/100
 - 0s - loss: 1558.7623
Epoch 2/100
 - 0s - loss: 1541.9069
Epoch 3/100
 - 0s - loss: 1525.1993
Epoch 4/100
 - 0s - loss: 1507.9767
Epoch 5/100
 - 0s - loss: 1490.5479
Epoch 6/100
 - 0s - loss: 1471.8432
Epoch 7/100
 - 0s - loss: 1452.4452
Epoch 8/100
 - 0s - loss: 1431.7661
Epoch 9/100
 - 0s - loss: 1409.9033
Epoch 10/100
 - 0s - loss: 1386.6817
Epoch 11/100
 - 0s - loss: 1362.3670
Epoch 12/100
 - 0s - loss: 1336.7583
Epoch 13/100
 - 0s - loss: 1310.1525
Epoch 14/100
 - 0s - loss: 1282.4341
Epoch 15/100
 - 0s - loss: 1253.7841
Epoch 16/100
 - 0s - loss: 1223.9646
Epoch 17/100
 - 0s - loss: 1193.6209
Epoch 18/100
 - 0s - loss: 1162.2114
Epoch 19/100
 - 0s - loss: 1129.8975
Epoch 20/100
 - 0s - loss: 1097.0106
Epoch 21/100
 - 0s - loss: 1063.8014
Epoch 22/100
 - 0s - loss: 1029.8292
Epoch 23/100
 - 0s - loss: 995.6159
Epoch 24/100
 - 0s - loss: 961.2901
Epoch 25/100
 - 0s - loss: 926.9028
Epoch 26/100
 - 0s - loss: 892.4486
Epoch 27/100
 - 0s - loss: 858.0216
Epoch 28/100
 -

In [29]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 0s 163us/step


147.86256591401826

In [30]:
from sklearn.metrics import mean_squared_error
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

147.86256381990808 0.0


In [32]:
total_mean_squared_errors = 100
epochs = 100
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors with normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 41.10579025861129
MSE 2: 48.21922678777701
MSE 3: 40.94471154320973
MSE 4: 44.252308509110634
MSE 5: 41.98518453061002
MSE 6: 45.02698482278867
MSE 7: 49.35161420211051
MSE 8: 39.336823485044214
MSE 9: 42.91802163725918
MSE 10: 42.378384994457456
MSE 11: 41.98241460361913
MSE 12: 39.69295826390337
MSE 13: 47.29281085363098
MSE 14: 45.603677521245764
MSE 15: 42.25389400346379
MSE 16: 35.56210430848946
MSE 17: 42.33768720534241
MSE 18: 40.263252085466604
MSE 19: 38.912179841964374
MSE 20: 41.487168901560764
MSE 21: 39.49610216949364
MSE 22: 38.757539409649794
MSE 23: 35.67428663556244
MSE 24: 39.24798549417539
MSE 25: 42.48270711312402
MSE 26: 41.8259110682219
MSE 27: 38.47235432103228
MSE 28: 40.21650153224908
MSE 29: 44.635929379262585
MSE 30: 42.79555289259235
MSE 31: 39.486158352453735
MSE 32: 36.898692899537316
MSE 33: 36.913294140960794
MSE 34: 43.34098049274926
MSE 35: 40.19362252738483
MSE 36: 45.09442260273066
MSE 37: 38.70262695670514
MSE 38: 43.32538073888489
MSE 39: 40